# Descarga de datos de hospitalizados desde Sanidad

## Objetivo

Vamos a hacer una lectura de los datos de hospitalizados publicados en los informes diarios de datos de Covid19 publicados por Sanidad. Este es un [ejemplo](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf) en página 3,  publicado del 4 de Diciembre 2020.
  
Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)

versión definitiva documentada, para ser publicar en el repo

[@walyt](https://twitter.com/walyt)

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [1]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import fitz
# https://pypi.org/project/PyMuPDF/#description

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [2]:
fuente_ficheros_pdf_sanidad = '/Users/mharias/documents/proyectos/covid/sanidad_fallecidos/pdfs_sanidad/'
dir_csv = '/Users/mharias/documents/proyectos/covid/sanidad_hospital/data/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_{:02d}_COVID-19.pdf'

## Descarga de los ficheros de Sanidad

### Función de para descargar un fichero pdf, copiada del script de [@alfonsotwr](https://github.com/alfonsotwr/snippets/tree/master/covidia-cam)

In [3]:
def descarga(url,num):
    print('Descargando:', url)
    fn=fuente_ficheros_pdf_sanidad+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return True

### Descarga de un rango o de un solo pdf

Descargamos el rango completo en el caso de que sea la primera vez. Arrancamos con el 77, pues no pude descifrar el formato de los pdf anteriores.

In [4]:
for i in range(287,297):
    descarga(URL_reg.format(i),i)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_287_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_288_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_289_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_290_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_291_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_292_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_293_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccay

KeyboardInterrupt: 

### Descarga de un solo fichero

Como ejemplo 266 corresponde al Viernes 4 Diciembre 2020, grabamos el fichero en el directorio local con el nº de orden del documento.

In [5]:
descarga(URL_reg.format(316),316)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_316_COVID-19.pdf


True

## Parseado y preparación del pandas

In [6]:
columnas = ['fecha','ccaa','Hosp_Totales','Hosp_7días','UCI_Totales','UCI_7dias','Hosp_actual','%CamasCovid',
        'UCI_actual','%CamasUCI','Ingresos_24horas','Altas_24horas']
datos = pd.DataFrame(columns=columnas)

fichero auxiliar que mapea nº de informes con fechas....Mejora: automatizarlo

In [7]:
claves={}
indice=76
rango= np.arange(datetime(2020,4,16),datetime.today(),timedelta(days=1)).astype(datetime)
for day in rango:
    if not (((day>datetime(2020,6,30))&(day.weekday() in [5,6])) |\
            (day==datetime(2020,12,6)) | (day==datetime(2020,12,8)) | (day==datetime(2020,12,25)) | \
           (day==datetime(2021,1,1)) | (day==datetime(2021,1,6))):
        #print (day,i,)
        indice+=1
        claves[indice]=day.strftime('%-d/%-m/%Y')

In [8]:
comunidades=['Andalucía','Aragón','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
             'Castilla y León','Cataluña','Ceuta','C. Valenciana','Extremadura','Galicia',
             'Madrid','Melilla','Murcia','Navarra','País Vasco','La Rioja']

# antes 235

In [9]:
rango_informes = np.concatenate([np.arange(88,137),np.arange(138,234)])
#rango_informes = np.concatenate([np.arange(88,90)])
opt='text'
for informe in rango_informes:
    fn=fuente_ficheros_pdf_sanidad+str(informe)+'.pdf'
    doc = fitz.open(fn)
    page = doc.load_page(1)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
        regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
        regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
        cadena = re.search(regexp,text).group(0)  
        #c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).replace('\n','').split()
        c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
        
        datos = datos.append({'fecha':claves[informe],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4},ignore_index=True)
        
        
        #print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        #print (cadena)

## desde 235

In [10]:
rango = np.concatenate([np.arange(235,266),np.arange(267,317)])
opt='text'
for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina = 3 if i==282 else 2
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{} '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){4})'+\
        '(\n \n(\w+[ .\w]*) )((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '(((\n\d*[,.]?\d+¥? )){2})'
        pattern = re.compile(regexp)
        c1,c2,c3,c4 = re.search(pattern,text).group(1).replace('\n','').split()
        c5 = re.search(pattern,text).group(6).replace('\n','')
        c6 = re.search(pattern,text).group(8).replace('\n','')
        c7 = re.search(pattern,text).group(9).replace('\n','')
        c8 = re.search(pattern,text).group(11).replace('\n','')
        c9, c10 = re.search(pattern,text).group(13).replace('\n','').split()
        
        #print (i,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6,'-',c7,'-',c8,'-',c9,'-',c10)
        datos = datos.append({'fecha':claves[i],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4,'Hosp_actual':c5,
                                                 '%CamasCovid':c6,'UCI_actual':c7,
                                                 '%CamasUCI':c8,'Ingresos_24horas':c9,
                                                 'Altas_24horas':c10},ignore_index=True)
        

In [11]:
datos

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,27/4/2020,Andalucía,5.768,20,720,4,NaN,NaN,NaN,NaN,NaN,NaN
1,27/4/2020,Aragón,2.403,8,257,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27/4/2020,Asturias,1.814,24,137,3,NaN,NaN,NaN,NaN,NaN,NaN
3,27/4/2020,Baleares,1.062,0,166,0,NaN,NaN,NaN,NaN,NaN,NaN
4,27/4/2020,Canarias,881,1,171,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4289,19/2/2021,Melilla,474,6,72,1,25,"13,74%",5,"29,41%",2,2
4290,19/2/2021,Murcia,8.362,106,1.072,15,377,"9,13%",109,"22,43%",25,39
4291,19/2/2021,Navarra,5.097,54,440,6,112,"5,77%",23,"17,56%",7,6
4292,19/2/2021,País Vasco,12.215,83,1.023,12,664,"13,21%",152,"33,26%",68,85


In [12]:
datos.to_csv(dir_csv+'datos_sanidad_hosp.csv',index=False,index_label=False)